## Unidad 6: Clasificación
### Actividad 11

## Descripción de Actividades

* En esta sesión trabajaremos con un archivo `.csv` donde se registraron las características de una serie de individuos para analizar la propensidad a tener alguna enfermedad coronaria.

* `sbp`: Presión Sanguínea Sistólica.
* `tobacco`: Promedio tabaco consumido por día.
* `ldl`: Lipoproteína de baja densidad.
* `adiposity`: Adiposidad.
* `famhist`: Antecedentes familiares de enfermedades cardiácas. (Binaria)
* `types`: Personalidad tipo A
* `obesity`: Obesidad.
* `alcohol`: Consumo actual de alcohol.
* `age`: edad.
* `chd`: Enfermedad coronaria. (dummy)


## Ejercicio 1: Preparar el ambiente de trabajo

* Importe los módulos básicos para el análisis de datos
* Importe `seaborn`.
* Importe `statsmodels`
* Importe `LogisticRegression`, `train_test_split`, `StandardScaler`, `classification_report` y `roc_curve` de `sklearn`

In [1]:
import pandas as pd
import numpy as np
import seaborn
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve

In [2]:
df = pd.read_csv('southafricanheart.csv')
df = df.drop(columns="Unnamed: 0")

In [3]:
df.describe()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd
count,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000
mean,138.326840,3.635649,4.740325,25.406732,53.103896,26.044113,17.044394,42.816017,0.346320
std,20.496317,4.593024,2.070909,7.780699,9.817534,4.213680,24.481059,14.608956,0.476313
min,101.000000,0.000000,0.980000,6.740000,13.000000,14.700000,0.000000,15.000000,0.000000
25%,124.000000,0.052500,3.282500,19.775000,47.000000,22.985000,0.510000,31.000000,0.000000
50%,134.000000,2.000000,4.340000,26.115000,53.000000,25.805000,7.510000,45.000000,0.000000
75%,148.000000,5.500000,5.790000,31.227500,60.000000,28.497500,23.892500,55.000000,1.000000
max,218.000000,31.200000,15.330000,42.490000,78.000000,46.580000,147.190000,64.000000,1.000000



## Ejercicio 2

* A continuación se presenta el siguiente modelo a estimar:

![alttext](eq1.gif)

* Para ello ejecute los siguientes pasos:
    1. Recodifique `famhist` a dummy, asignando 1 a la categoría minoritaria.
    - Utilice `smf.logit` para estimar el modelo.

* Implemente una función `inverse_logit` que realize el mapeo de log-odds a probabilidad.

* Con el modelo estimado, responda lo siguiente:
    1. ¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria?
    - ¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria?
    - ¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?

* Replique el modelo con `smf.ols` y comente las similitudes entre los coeficientes estimados. 
 - _tip_: Utilice <img src="https://latex.codecogs.com/svg.latex?\beta/4" title="\beta/4" />


In [4]:
df['famhist'].value_counts()

Absent     270
Present    192
Name: famhist, dtype: int64

In [5]:
df['famhist_bin'] = np.where(df['famhist'] == 'Present', 1, 0)

In [6]:
m1_logit = smf.logit('chd ~ famhist_bin',df).fit()

Optimization terminated successfully.
         Current function value: 0.608111
         Iterations 5


In [7]:
m1_logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      460
Method:                           MLE   Df Model:                            1
Date:                Sat, 22 Dec 2018   Pseudo R-squ.:                 0.05740
Time:                        22:09:56   Log-Likelihood:                -280.95
converged:                       True   LL-Null:                       -298.05
                                        LLR p-value:                 4.937e-09
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -1.1690      0.143     -8.169      0.000      -1.449      -0.889
famhist_bin     1.1690      0.203      5.751      0.000       0.771       1.567
===============================================================================
"""

In [8]:
m1_logit.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.057     
Dependent Variable: chd              AIC:              565.8944  
Date:               2018-12-22 22:09 BIC:              574.1655  
No. Observations:   462              Log-Likelihood:   -280.95   
Df Model:           1                LL-Null:          -298.05   
Df Residuals:       460              LLR p-value:      4.9371e-09
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     5.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept     -1.1690    0.1431  -8.1687  0.0000  -1.4495  -0.8885
famhist_bin    1.1690    0.2033   5.7514  0.0000   0.7706   1.5674
=================================================================

"""

In [9]:
def log_odds(dataframe, variable, modelo, estadobin = 1):
    estimate_y = modelo.params['Intercept'] + (modelo.params[variable] * estadobin)
    return estimate_y

In [10]:
log_odds(df,'famhist_bin',m1_logit,0)

-1.1689930854299095

In [11]:
log_odds(df,'famhist_bin',m1_logit,1)

-2.220446049250313e-16

In [12]:
def inverse_logit(x):
    return 1 / (1 + np.exp(-x))

In [13]:
round(inverse_logit(log_odds(df,'famhist_bin',m1_logit,0)),2)

0.24

In [14]:
round(inverse_logit(log_odds(df,'famhist_bin',m1_logit,1)),2)

0.5

La probabilidad de un sujeto que tiene antecedentes familiares de enfermedades cardiacas de sufrir una Enfermedad coronaria es de un 50%

La probabilidad de un sujeto que no tiene antecedentes familiares de enfermedades cardiacas de sufrir una Enfermedad coronaria es de un 24%

In [15]:
inverse_logit(log_odds(df,'famhist_bin',m1_logit,1))-inverse_logit(log_odds(df,'famhist_bin',m1_logit,0))

0.262962962962963

In [16]:
model1 = smf.ols('chd ~ famhist_bin', df)
model1 = model1.fit()
model1.params['famhist_bin']

0.2629629629629629

<img src="https://latex.codecogs.com/svg.latex?\beta/4" title="\beta/4" />

In [17]:
m1_logit.params['famhist_bin']/4

0.29224827135747733

In [18]:
round((m1_logit.params['famhist_bin']/4-model1.params['famhist_bin']),2)

0.03

La diferencia entre los coeficientes estimados es de aproximadamente 0.03 puntos percentuales, lo cual a priori presenta bastante similitud

## Ejercicio 3: Estimación completa

* Implemente un modelo con la siguiente forma

![alttext](eq2.gif)

* Depure el modelo manteniendo las variables con significancia estadística al 95%.
* Compare los estadísticos de bondad de ajuste entre ambos.
* Reporte de forma sucinta el efecto de las variables en el log-odds de tener una enfermedad coronaria.

In [19]:
def saturated_model(df, dependent, estimation=smf.logit,fit_model=True):
    """
    saturated_model - returns a saturated model

    @parameters:
        - df: a `pandas.core.frame.DataFrame` object.
        - dependent: String. Name of the dependent variable to be modelled.
        - estimation: Method. A `statsmodels` class estimator
        - fit_model: Bool. Whether the returned model should be fitter or not. Default: True

    @returns:
        - A `smf` model.

    """
    # seleccionamos todas las variables que no sean la dependiente
    tmp_vars = "+".join(df.columns.drop(dependent))
    # generamos la estimación del modelo
    tmp_model = estimation(dependent+ '~ '+ tmp_vars, df)
    # si fit model es verdadero, podemos realizar el fit de forma automática
    if fit_model is True:
        tmp_model = tmp_model.fit()
    # recuerden retornar el objeto :)
    return tmp_model

In [20]:
df = df.drop(columns="famhist")

In [21]:
modelo_saturado = saturated_model(df, 'chd')

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6


In [22]:
modelo_saturado.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.208     
Dependent Variable: chd              AIC:              492.1400  
Date:               2018-12-22 22:09 BIC:              533.4957  
No. Observations:   462              Log-Likelihood:   -236.07   
Df Model:           9                LL-Null:          -298.05   
Df Residuals:       452              LLR p-value:      2.0548e-22
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept     -6.1507    1.3083  -4.7015  0.0000  -8.7149  -3.5866
sbp            0.0065    0.0057   1.1350  0.2564  -0.0047   0.0177
tobacco        0.0794    0.0266   2.9838  0.0028   0.0272   0.1315
ldl            0.1739    0.0597   2.9152  0.0036   0.0570   0.2909
adiposity      0.0186    0.0293   0.6346  0.5257  -0.0388   0.0760
typea          0.0396    0.0123   3.2138  0.0013   0.0154   0.0637
obesity       -0.0629    0.0442  -1.4218  0.1551  -0.1496   0.0238
alcohol        0.0001    0.0045   0.0271  0.9784  -0.0087   0.0089
age            0.0452    0.0121   3.7285  0.0002   0.0215   0.0690
famhist_bin    0.9254    0.2279   4.0605  0.0000   0.4787   1.3720
=================================================================

"""

In [53]:
modelo_saturado.params[modelo_saturado.pvalues < 0.05]

Intercept     -6.150721
tobacco        0.079376
ldl            0.173924
typea          0.039595
age            0.045225
famhist_bin    0.925370
dtype: float64

In [24]:
df_depurado = df.loc[:,['tobacco','ldl','typea','age','famhist_bin', 'chd']]

In [25]:
modelo_saturado_2 = saturated_model(df_depurado, 'chd')

Optimization terminated successfully.
         Current function value: 0.514811
         Iterations 6


In [26]:
modelo_saturado_2.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.202     
Dependent Variable: chd              AIC:              487.6856  
Date:               2018-12-22 22:09 BIC:              512.4990  
No. Observations:   462              Log-Likelihood:   -237.84   
Df Model:           5                LL-Null:          -298.05   
Df Residuals:       456              LLR p-value:      2.5537e-24
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept     -6.4464    0.9209  -7.0004  0.0000  -8.2513  -4.6416
tobacco        0.0804    0.0259   3.1057  0.0019   0.0297   0.1311
ldl            0.1620    0.0550   2.9470  0.0032   0.0543   0.2697
typea          0.0371    0.0122   3.0505  0.0023   0.0133   0.0610
age            0.0505    0.0102   4.9442  0.0000   0.0305   0.0705
famhist_bin    0.9082    0.2258   4.0228  0.0001   0.4657   1.3507
=================================================================

"""

### Estadísticos de bondad de ajuste entre ambos modelos
#### Para el modelo saturado

`Log-Likelihood`: -236.07

`LL-Null`: -298.05

`LLR p-value`: 2.0548e-22


#### Para el modelo saturado depurado

`Log-Likelihood`: -237.84

`LL-Null`: -298.05

`LLR p-value`: 2.5537e-24

El estadístico de `LLR p-value` indica cuan cercano está el modelo para explicar la variable estudiada, por lo cual, el `modelo saturado depurado` explica de mejor manera el comportamiento de la variable `chd`

Al analizar los coeficientes del `modelo_saturado_depurado` se puede apreciar que la variable más significativa para estudiar `chd` es el `famhist_bin`, la cual explica que efecto tener Antecedentes familiares de enfermedades cardiacas afecta en 0.9 veces en el log-odds de tener una enfermedad coronaria.

## Ejercicio 4: Estimación de perfiles

* A partir del modelo depurado, genere las estimaciones en log-odds y posteriormente transfórmelas a probabilidades con `inverse_logit`. Los perfiles a estimar son los siguientes:

1. La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra.
- La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja densidad, __manteniendo todas las demás características constantes__.
- La probabilidad de tener una enfemerdad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad, __manteniendo todas las demás características constantes__.

In [27]:
modelo_saturado_2.params.drop('Intercept')

tobacco        0.080375
ldl            0.161992
typea          0.037115
age            0.050460
famhist_bin    0.908175
dtype: float64

In [28]:
#calcular log-odds
estimate_y = modelo_saturado_2.params['tobacco'] * df_depurado['tobacco'].mean()
estimate_y += modelo_saturado_2.params['ldl'] * df_depurado['ldl'].mean()
estimate_y += modelo_saturado_2.params['typea'] * df_depurado['typea'].mean()
estimate_y += modelo_saturado_2.params['age'] * df_depurado['age'].mean()
estimate_y += modelo_saturado_2.params['famhist_bin'] * df_depurado['famhist_bin'].mean()
estimate_y += modelo_saturado_2.params['Intercept']

In [29]:
inverse_logit(estimate_y)

0.29370927481586945

Un individuo con características similares a la muestra depurada tiene aproximadamente un 29% de probabilidad de sufrir una enfermedad coronaria.

In [30]:
#calcular log-odds
estimate_y = modelo_saturado_2.params['tobacco'] * df_depurado['tobacco'].mean()
estimate_y += modelo_saturado_2.params['ldl'] * df_depurado['ldl'].max()
estimate_y += modelo_saturado_2.params['typea'] * df_depurado['typea'].mean()
estimate_y += modelo_saturado_2.params['age'] * df_depurado['age'].mean()
estimate_y += modelo_saturado_2.params['famhist_bin'] * df_depurado['famhist_bin'].mean()
estimate_y += modelo_saturado_2.params['Intercept']

In [31]:
inverse_logit(estimate_y)

0.6980443104466211

Un individuo con altos niveles de lipoproteína de baja densidad de la muestra depurada tiene aproximadamente un 70% de probabilidad de sufrir una enfermedad coronaria.

In [32]:
#calcular log-odds
estimate_y = modelo_saturado_2.params['tobacco'] * df_depurado['tobacco'].mean()
estimate_y += modelo_saturado_2.params['ldl'] * df_depurado['ldl'].min()
estimate_y += modelo_saturado_2.params['typea'] * df_depurado['typea'].mean()
estimate_y += modelo_saturado_2.params['age'] * df_depurado['age'].mean()
estimate_y += modelo_saturado_2.params['famhist_bin'] * df_depurado['famhist_bin'].mean()
estimate_y += modelo_saturado_2.params['Intercept']

In [33]:
inverse_logit(estimate_y)

0.18443595575404653

Un individuo con bajos niveles de lipoproteína de baja densidad de la muestra depurada tiene aproximadamente un 18% de probabilidad de sufrir una enfermedad coronaria.